# 🎬 Avatar Renderer MCP - Google Colab Demo (Fixed)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ruslanmv/avatar-renderer-mcp/blob/main/demo_colab.ipynb)

**Transform static images into dynamic AI-powered avatars with realistic expressions and voice synchronization**

🔧 **This notebook includes all critical fixes for Colab compatibility**

---

**⚡ GPU Recommended**: Runtime → Change runtime type → GPU (T4)

**🌐 Website:** [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)

## 1️⃣ Environment Check & System Dependencies

In [ ]:
# Check environment
import sys
import os
from pathlib import Path

print("🔍 Environment Information")
print("="*60)
print(f"Python Version: {sys.version}")
py_version = sys.version_info
if py_version < (3, 10):
    print("❌ ERROR: Python 3.10+ required")
    raise RuntimeError("Python 3.10+ required. Please use a compatible runtime.")
else:
    print(f"✅ Python {py_version.major}.{py_version.minor} compatible")

# Check Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("⚠️  Not in Colab")

# Check GPU
import torch
cuda_available = torch.cuda.is_available()
print(f"\nGPU Available: {cuda_available}")
if cuda_available:
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU. Processing will be slower.")
print("="*60)

In [ ]:
# Install system dependencies
print("📦 Installing system dependencies...\n")
!apt-get update -qq && apt-get install -y -qq ffmpeg libsm6 libxext6 libxrender-dev libgomp1 git git-lfs > /dev/null 2>&1
!ffmpeg -version | head -3
print("\n✅ System dependencies installed!")

## 2️⃣ Clone Repository

In [ ]:
# Clone repository
REPO_DIR = Path("/content/avatar-renderer-mcp")

if REPO_DIR.exists():
    print("📂 Repository exists, pulling latest...")
    !cd {REPO_DIR} && git pull -q
else:
    print("📥 Cloning repository...")
    !git clone -q https://github.com/ruslanmv/avatar-renderer-mcp.git {REPO_DIR}

os.chdir(REPO_DIR)
print(f"✅ Working directory: {os.getcwd()}")

## 3️⃣ Install Python Dependencies

**CRITICAL FIX**: We install without forcing torch reinstall to preserve Colab's optimized GPU-enabled torch.

In [ ]:
# Install Python dependencies (FIX: use --no-deps for core package to avoid torch conflicts)
print("📦 Installing Python dependencies...")
print("This preserves Colab's optimized torch installation.\n")

# Install the package structure without dependencies first
!pip install -q --no-deps -e .

# Then install only the safe dependencies (avoiding torch/torchvision reinstall)
!pip install -q fastapi uvicorn python-multipart pydantic pydantic-settings python-dotenv
!pip install -q diffusers transformers huggingface-hub safetensors accelerate
!pip install -q gfpgan librosa soundfile
!pip install -q gdown opencv-python imageio moviepy
!pip install -q requests click rich
!pip install -q ffmpeg-python pandas  # CRITICAL FIX: Required by first-order-model

print("\n✅ Python dependencies installed!")
import torch  # Import torch to verify version
print(f"✅ Torch version: {torch.__version__} (Colab's version preserved)")

## 4️⃣ Install External Dependencies

**CRITICAL FIX**: We clone repos but DON'T install their requirements.txt (which would break torch)

In [ ]:
# Setup external dependencies directory
EXT_DEPS_DIR = REPO_DIR / "external_deps"
EXT_DEPS_DIR.mkdir(exist_ok=True)

print("📦 Installing external dependencies...\n")

# SadTalker
sadtalker_dir = EXT_DEPS_DIR / "SadTalker"
if not sadtalker_dir.exists():
    print("📥 Cloning SadTalker...")
    !git clone -q --depth=1 https://github.com/OpenTalker/SadTalker.git {sadtalker_dir}
    # FIX: Don't install requirements.txt - install only safe deps
    !pip install -q yacs pyyaml
else:
    print("✓ SadTalker already installed")

# First Order Motion Model
fomm_dir = EXT_DEPS_DIR / "first-order-model"
if not fomm_dir.exists():
    print("📥 Cloning First Order Motion Model...")
    !git clone -q --depth=1 https://github.com/AliaksandrSiarohin/first-order-model.git {fomm_dir}
else:
    print("✓ FOMM already installed")

# Wav2Lip
wav2lip_dir = EXT_DEPS_DIR / "Wav2Lip"
if not wav2lip_dir.exists():
    print("📥 Cloning Wav2Lip...")
    !git clone -q --depth=1 https://github.com/Rudrabha/Wav2Lip.git {wav2lip_dir}
    # FIX: Don't install requirements.txt - deps already covered
else:
    print("✓ Wav2Lip already installed")

print("\n✅ External dependencies installed!")

## 5️⃣ Download Model Checkpoints

**CRITICAL FIX**: Using the official download script to get the COMPLETE model set

In [ ]:
# Setup models directory
MODELS_DIR = Path("/content/models")

# Uncomment to use Google Drive (persistent storage)
# from google.colab import drive
# drive.mount('/content/drive')
# MODELS_DIR = Path("/content/drive/MyDrive/avatar-renderer-models")

MODELS_DIR.mkdir(exist_ok=True, parents=True)
os.environ['MODEL_ROOT'] = str(MODELS_DIR)

print(f"📁 Models directory: {MODELS_DIR}")
print(f"Storage: {'Google Drive (persistent)' if 'drive' in str(MODELS_DIR) else 'Colab storage (temporary)'}")

In [ ]:
# Download models using the official script (FIX: ensures all required models are downloaded)
print("📥 Downloading model checkpoints using official script...")
print("This ensures all required models are downloaded correctly.\n")

# Use the repo's official download script
!bash {REPO_DIR}/scripts/download_models.sh {MODELS_DIR}

# Verify critical models exist
required_models = [
    MODELS_DIR / "sadtalker" / "sadtalker.pth",
    MODELS_DIR / "wav2lip" / "wav2lip_gan.pth",
    MODELS_DIR / "gfpgan" / "GFPGANv1.3.pth",  # FIX: v1.3 not v1.4
]

print("\n🔍 Verifying model downloads:")
all_exist = True
for model_path in required_models:
    if model_path.exists():
        size_mb = model_path.stat().st_size / (1024*1024)
        print(f"  ✅ {model_path.name} ({size_mb:.1f} MB)")
    else:
        print(f"  ❌ {model_path.name} - NOT FOUND")
        all_exist = False

if all_exist:
    print("\n✅ All required models downloaded successfully!")
else:
    print("\n⚠️  Some models missing. Real-time mode may still work.")

## 6️⃣ Configure Environment & MCP Server

**CRITICAL FIX**: Setting correct PYTHONPATH and EXT_DEPS_DIR

In [ ]:
# Configure environment variables (FIX: proper PYTHONPATH)
import sys
import torch  # FIX: Import torch explicitly

print("⚙️ Configuring environment...\n")

# Set environment variables
env_vars = {
    'MODEL_ROOT': str(MODELS_DIR),
    'EXT_DEPS_DIR': str(EXT_DEPS_DIR),  # FIX: Add EXT_DEPS_DIR
    'LOG_LEVEL': 'INFO',
    'TMP_DIR': '/tmp',
    'CUDA_VISIBLE_DEVICES': '0' if torch.cuda.is_available() else '',
}

for key, value in env_vars.items():
    os.environ[key] = value
    print(f"✓ {key}={value[:60]}{'...' if len(value) > 60 else ''}")

# FIX: Add external deps to Python path correctly
paths_to_add = [
    str(REPO_DIR),
    str(EXT_DEPS_DIR),
    str(EXT_DEPS_DIR / "SadTalker"),
    str(EXT_DEPS_DIR / "Wav2Lip"),
    str(EXT_DEPS_DIR / "first-order-model"),
]

for path in paths_to_add:
    if path not in sys.path:
        sys.path.insert(0, path)

print(f"\n✅ Python path configured with {len(paths_to_add)} directories")
print("✅ Environment ready!")

In [ ]:
# Verify imports
print("🧪 Verifying installation...\n")

errors = []

try:
    from app.pipeline import render_pipeline
    print("✅ Pipeline module")
except Exception as e:
    print(f"❌ Pipeline module: {e}")
    errors.append(str(e))

try:
    from app.settings import settings
    print("✅ Settings module")
    print(f"   Model root: {settings.MODEL_ROOT}")
except Exception as e:
    print(f"❌ Settings module: {e}")
    errors.append(str(e))

# Check key dependencies
deps = ['torch', 'torchvision', 'diffusers', 'transformers', 'librosa']
for dep in deps:
    try:
        __import__(dep)
        print(f"✅ {dep}")
    except ImportError as e:
        print(f"❌ {dep}: {e}")
        errors.append(f"{dep}: {e}")

# Check gfpgan separately (may have compatibility issues with newer torchvision)
try:
    import gfpgan
    print("✅ gfpgan")
except ImportError as e:
    print(f"⚠️  gfpgan: {e}")
    print("   (Face enhancement may be unavailable, but core features will work)")

if not errors:
    print("\n✅ All components verified!")
    print("🎬 Ready to generate avatars!")
else:
    print(f"\n⚠️  {len(errors)} errors found")
    print("Some features may not work. Try re-running installation cells.")

## 7️⃣ Generate Hello World Avatar

In [ ]:
# Setup output directory
OUTPUT_DIR = Path("/content/avatar_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# Check test assets
test_image = REPO_DIR / "tests/assets/alice.png"
test_audio = REPO_DIR / "tests/assets/hello.wav"

print("📁 Output directory:", OUTPUT_DIR)
print("\n🎨 Test assets:")

if test_image.exists():
    print(f"  ✅ Image: {test_image.name}")
    from IPython.display import Image, display
    display(Image(filename=str(test_image), width=300))
else:
    print(f"  ❌ Image not found")

if test_audio.exists():
    print(f"  ✅ Audio: {test_audio.name}")
    from IPython.display import Audio, display
    display(Audio(filename=str(test_audio)))
else:
    print(f"  ❌ Audio not found")

In [ ]:
# Generate avatar
import time
from IPython.display import Video, display
from app.pipeline import render_pipeline

output_video = OUTPUT_DIR / "hello_world.mp4"

print("🎬 Generating 'Hello World' Avatar")
print("="*60)
print(f"Image: {test_image.name}")
print(f"Audio: {test_audio.name}")
print(f"Quality: auto (adapts to available hardware)")
print("="*60)
print("\n⏳ Processing...\n")

start_time = time.time()

try:
    result = render_pipeline(
        face_image=str(test_image),
        audio=str(test_audio),
        out_path=str(output_video),
        quality_mode="auto"
    )
    
    elapsed = time.time() - start_time
    
    print(f"\n{'='*60}")
    print("✅ SUCCESS!")
    print(f"{'='*60}")
    print(f"⏱️  Time: {elapsed:.1f}s")
    print(f"📁 File: {result}")
    print(f"📊 Size: {Path(result).stat().st_size / (1024*1024):.2f} MB")
    print(f"{'='*60}\n")
    
    print("🎥 Your AI Avatar:")
    display(Video(result, width=640, embed=True))
    
except Exception as e:
    elapsed = time.time() - start_time
    print(f"\n{'='*60}")
    print("❌ ERROR")
    print(f"{'='*60}")
    print(f"Error: {str(e)}")
    print(f"Time: {elapsed:.1f}s")
    print(f"\n🔧 Troubleshooting:")
    print("1. Check models downloaded correctly (Section 5)")
    print("2. Verify test assets exist (Section 7)")
    print("3. Try quality_mode='real_time' for CPU systems")
    print(f"{'='*60}\n")
    
    import traceback
    print("\n📋 Full error:")
    traceback.print_exc()

## 8️⃣ Create Custom Avatars

Upload your own files or use different quality modes

In [ ]:
# Upload custom files
if IN_COLAB:
    from google.colab import files
    import shutil
    
    UPLOAD_DIR = Path("/content/uploads")
    UPLOAD_DIR.mkdir(exist_ok=True)
    
    print("📤 Upload your files (image + audio):")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        dst = UPLOAD_DIR / filename
        shutil.move(filename, str(dst))
        print(f"✅ {filename}")
else:
    print("⚠️  File upload only works in Colab")
    UPLOAD_DIR = REPO_DIR / "tests/assets"

In [ ]:
# Generate from uploaded files
image_files = list(UPLOAD_DIR.glob("*.png")) + list(UPLOAD_DIR.glob("*.jpg"))
audio_files = list(UPLOAD_DIR.glob("*.wav")) + list(UPLOAD_DIR.glob("*.mp3"))

if image_files and audio_files:
    custom_output = OUTPUT_DIR / "custom_avatar.mp4"
    
    print(f"🎬 Generating from: {image_files[0].name} + {audio_files[0].name}")
    
    result = render_pipeline(
        face_image=str(image_files[0]),
        audio=str(audio_files[0]),
        out_path=str(custom_output),
        quality_mode="auto"
    )
    
    print("\n✅ Custom avatar generated!")
    display(Video(result, width=640, embed=True))
else:
    print("⚠️  Upload both image and audio files")

## 9️⃣ Download Generated Videos

In [ ]:
# Download videos
if IN_COLAB:
    from google.colab import files
    
    videos = list(OUTPUT_DIR.glob("*.mp4"))
    
    if videos:
        print("📥 Available videos:")
        for video in videos:
            size_mb = video.stat().st_size / (1024*1024)
            print(f"  📹 {video.name} ({size_mb:.2f} MB)")
        
        print("\n💾 Downloading...")
        for video in videos:
            files.download(str(video))
            print(f"✅ {video.name}")
    else:
        print("No videos found. Generate some first!")
else:
    print("⚠️  Download only works in Colab")
    print(f"Videos saved to: {OUTPUT_DIR}")

## 🎉 Summary

### What Was Fixed

✅ **Python 3.10+ compatibility** - Updated pyproject.toml

✅ **Dependency conflicts resolved** - Preserved Colab's torch, loosened version constraints

✅ **External repos handled safely** - Cloned without breaking torch installation

✅ **Complete model downloads** - Used official script for all required models

✅ **GFPGAN version standardized** - v1.3 across all components

✅ **Import issues fixed** - Dynamic loading for SadTalker/Wav2Lip/FOMM

✅ **PYTHONPATH configured** - All external deps properly added

✅ **Missing imports added** - torch imported where needed

✅ **gdown installed** - Available when needed

### Performance Tips

- **GPU**: Enable for 5-10x speedup
- **Real-time mode**: Fast, works on CPU
- **High-quality mode**: Best results, requires GPU
- **Google Drive**: Mount for persistent model storage

### Support

- **Website**: [https://avatar-renderer-mcp.vercel.app/](https://avatar-renderer-mcp.vercel.app/)
- **Repository**: [https://github.com/ruslanmv/avatar-renderer-mcp](https://github.com/ruslanmv/avatar-renderer-mcp)
- **Issues**: [GitHub Issues](https://github.com/ruslanmv/avatar-renderer-mcp/issues)

---

*Made with ❤️ by [Ruslan Magana Vsevolodovna](https://ruslanmv.com)*